# Installing and Importing

In [ ]:
!pip install tavily-python


In [ ]:
!pip install langchain langgraph tavily sentence-transformers faiss-cpu


In [ ]:
!pip install "transformers==4.31.0"


In [ ]:
!pip install tokenizers==0.13.3


In [ ]:
!pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --force-reinstall


In [ ]:
import os
os.environ['TAVILY_API_KEY'] = 'Api key'
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
from sentence_transformers import CrossEncoder
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate


In [ ]:

!pip install transformers[sentencepiece,torch]
!pip install sentence-transformers


In [ ]:
!pip install langchain-openai


## Working of Tavily

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Step 1: Pass your Tavily API Key here
search_tool = TavilySearchResults(
    tavily_api_key="Tavily API Key"  
)

# Step 2: Write your query
query = "what is news in india?"

# Step 3: Perform the search
result = search_tool.invoke({"query": query})

# Step 4: Print the result
print("\nSearch Results:")
print(result)


## Main:

In [ ]:


# Set API Keys
os.environ["TAVILY_API_KEY"] = "TAVILY KEY"
os.environ["OPENAI_API_KEY"] = "OPENAI KEY"


# Initialize Tools
search_tool = TavilySearchResults()
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
llm = ChatOpenAI(model="gpt-3.5-turbo")

# Step 1: Search Function
def search_fn(query):
    results = search_tool.invoke({"query": query})
    return results

# Step 2: Rerank Function
def rerank_fn(query, results):
    pairs = [(query, item["content"]) for item in results]
    scores = reranker.predict(pairs)
    reranked = [item for _, item in sorted(zip(scores, results), key=lambda x: x[0], reverse=True)]
    return reranked[:3]  # Top 3 results

# Step 3: Format + Tone Control
def format_fn(reranked_results):
    prompt = ChatPromptTemplate.from_template("""
Format the following search results into a clean, professional tone. 
Make it engaging but formal. Summarize the key points clearly.

{results}
    """)
    formatted = llm.invoke(prompt.format_prompt(results=reranked_results).to_messages())
    return formatted.content

# Step 4: Critic Agent (Quality Check)
def critic_fn(formatted_result):
    prompt = ChatPromptTemplate.from_template("""
Act as a quality critic. Does the following response meet these rules:
- Is it clear and professional?
- Is it factually consistent?
- Is it relevant to the original query?

Respond "PASS" if all good, otherwise "FAIL" with reason.

Response:
{response}
    """)
    critique = llm.invoke(prompt.format_prompt(response=formatted_result).to_messages())
    return critique.content

# Step 5: Main Logic (without langgraph)
def main_logic(query):
    search_results = search_fn(query)
    reranked = rerank_fn(query, search_results)
    formatted = format_fn(reranked)
    critique = critic_fn(formatted)

    if "FAIL" in critique:
        print(" Critic said FAIL. Re-running search...")
        return main_logic(query)
    else:
        print(" Critic said PASS. Final result ready!")
        print(formatted)
        return formatted 

# Step 6: Run Everything
user_query = input("Enter your search query: ")
final_result = main_logic(user_query)
